# Assignment 2 - Beer Recommendation System

### Group Members: Ohyoon Kwon (ok2364), Oluwaseun Ibitoye (oli63), Aditya Soni (as96926), Samuel Oh (io3279), Ambikha Maharaj (am84333), Ankita Kundra (ak44675)

In [372]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
import en_core_web_lg
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import string
from collections import Counter
stop_words = stopwords.words('english') 
import io
import warnings
warnings.filterwarnings('ignore')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ohyoo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ohyoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Task A - WebScraper.io
Extract around 6K reviews using WebScraper.io

In [373]:
# read the CSV file
df = pd.read_csv('beer_data.csv')
df = df.drop(columns=['web-scraper-order', 'web-scraper-start-url', 'beer-href'])
df.head()

,beer,product_review
0,Aaron,flipper2gv from Canada (QC)\n\n4.75/5 rDev +2...
1,Mocha Wednesday,Radcpa from Washington\n\n4.27/5 rDev -5.5%\n...
2,Mexican Cake - Maple Bourbon Barrel-Aged,mothman from Minnesota\n\n4.69/5 rDev +2%\nlo...
3,Saison Du Fermier,REVZEB from Illinois\n\n4.35/5 rDev -4.8%\nlo...
4,Stickee Monkee,gooddaydiablo from Texas\n\n4.6/5 rDev +2.4%\...


In [374]:
# clean the data - split product_review
df['product_review'] = df.product_review.str.split('\n\n', expand=False)

In [375]:
# extract ratings from product_review
lst = []
for i in range(len(df)):
    a = df.product_review[i][1].split('/')
    lst.append(a[0])
df['rating'] = lst
df.head()

,beer,product_review,rating
0,Aaron,"[flipper2gv from Canada (QC), 4.75/5 rDev +2....",4.75
1,Mocha Wednesday,"[Radcpa from Washington, 4.27/5 rDev -5.5%\nl...",4.27
2,Mexican Cake - Maple Bourbon Barrel-Aged,"[mothman from Minnesota, 4.69/5 rDev +2%\nloo...",4.69
3,Saison Du Fermier,"[REVZEB from Illinois, 4.35/5 rDev -4.8%\nloo...",4.35
4,Stickee Monkee,"[gooddaydiablo from Texas, 4.6/5 rDev +2.4%\n...",4.6


In [376]:
# extracting review text
def get_review(s):
    a = ''.join(s[2:-1])
    return a

In [377]:
# final table
df['product_review'] = df['product_review'].apply(get_review)
beer_df = df[['beer','product_review','rating']]

In [378]:
beer_df

,beer,product_review,rating
0,Aaron,A: Pours a deep murky brown color with a beige...,4.75
1,Mocha Wednesday,Brown appearance with slight tan head that dis...,4.27
2,Mexican Cake - Maple Bourbon Barrel-Aged,Tan head but not a lot. Dark brown/black color...,4.69
3,Saison Du Fermier,Simplistic beauty is how I would describe this...,4.35
4,Stickee Monkee,"2018 vintage, tasted on 2021.1.11-------------...",4.6
...,...,...,...
6222,Bomb!,"Silky smooth texture. Rich flavors of coffee, ...",4.88
6223,Mocha Wednesday,2016 Vintage.Poured with quite a bit of sedime...,4.51
6224,Emerald Grouper,"Drank from a 1 pint, 6 fl. oz bottle purchased...",4.12
6225,Ghost In The Machine,Pours a hazed gold with a thumb of eggshell co...,4.13


In [94]:
beer_df.to_csv('beer.csv')

# Task B
Word frequency analysis to find important attributes
Assume that a customer has specified three attributes of the product as being important to him or her.

In [379]:
df = pd.read_csv('beer.csv')
df.head()

,Unnamed: 0,beer,product_review,rating
0,0,Aaron,A: Pours a deep murky brown color with a beige...,4.75
1,1,Mocha Wednesday,Brown appearance with slight tan head that dis...,4.27
2,2,Mexican Cake - Maple Bourbon Barrel-Aged,Tan head but not a lot. Dark brown/black color...,4.69
3,3,Saison Du Fermier,Simplistic beauty is how I would describe this...,4.35
4,4,Stickee Monkee,"2018 vintage, tasted on 2021.1.11-------------...",4.60


In [380]:
# Cleaning
df = df.drop(['Unnamed: 0'], axis=1)
df = df.dropna()

# renaming columns
df = df.rename({'beer':'product_name', 'rating':'user_rating'}, axis=1)

In [381]:
# removing punctuations, transforming everything to lowercase, and tokenizing
df['clean_review'] = df['product_review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df['clean_review'] = df['clean_review'].apply(lambda x: x.lower())
df['clean_review'] = df['clean_review'].apply(word_tokenize).apply(set).apply(list)

# removing stopwords
def remove_stopwords(s):
    return [w for w in s if not w in stop_words]

df['clean_review'] = df['clean_review'].apply(remove_stopwords)

In [382]:
# stitching the words into a corpus
def stitch_words(comment_col):
    corpus = []
    for words in comment_col:
        corpus += words
    return corpus
corpus = stitch_words(df['clean_review'])

# getting the word frequencies
word_freq = nltk.FreqDist(corpus)

In [383]:
# extracting the top frequencies
word_freq.most_common()

[('head', 3234),
 ('beer', 2740),
 ('taste', 2096),
 ('one', 1657),
 ('dark', 1652),
 ('sweet', 1648),
 ('like', 1604),
 ('nose', 1594),
 ('chocolate', 1588),
 ('good', 1492),
 ('pours', 1468),
 ('carbonation', 1421),
 ('nice', 1408),
 ('well', 1380),
 ('black', 1361),
 ('body', 1360),
 ('bottle', 1343),
 ('light', 1316),
 ('finish', 1316),
 ('notes', 1277),
 ('vanilla', 1276),
 ('aroma', 1270),
 ('mouthfeel', 1247),
 ('white', 1246),
 ('medium', 1213),
 ('bit', 1195),
 ('smooth', 1185),
 ('flavor', 1160),
 ('bourbon', 1158),
 ('great', 1142),
 ('color', 1136),
 ('really', 1135),
 ('fruit', 1134),
 ('thick', 1128),
 ('coffee', 1085),
 ('little', 1081),
 ('poured', 1081),
 ('orange', 1053),
 ('flavors', 1032),
 ('overall', 1004),
 ('brown', 998),
 ('lacing', 982),
 ('barrel', 976),
 ('glass', 969),
 ('oak', 940),
 ('citrus', 939),
 ('feel', 925),
 ('bitterness', 913),
 ('much', 881),
 ('sweetness', 849),
 ('’', 845),
 ('creamy', 841),
 ('smell', 820),
 ('hazy', 813),
 ('malt', 810),
 ('

#### Three attributes of the product that are important to the customer: Fruity, Balanced, Robust

In [384]:
# this dataframe will be used for Task F
df_f = df.copy()

# this dataframe will be used for Task G
df_g = df.copy()

# Task C

In [385]:
# three attributes that we chose as input
attributes = ['fruity', 'balanced', 'robust']

# join the clean_review column
def join_words(comment):
    return " ".join(comment)
df['join_review'] = df['clean_review'].map(join_words)

# join the attributes into one string
input_attributes = " ".join(attributes)

In [386]:
# cosine similarity with bag of words
def bow_similarity(comments):
    documents = [input_attributes, comments]
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(documents)
    doc_term_matrix = sparse_matrix.todense()
    temp_df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names(), index=['x', 'y'])
    return cosine_similarity(temp_df, temp_df)[0, 1]

df["similarity_score"] = df['join_review'].apply(bow_similarity)

In [387]:
# create another version of the table for the output
df_c = df.copy()
df_c = df.drop(['clean_review', 'join_review', 'user_rating'], axis=1)

# sort values according to the similarity score
df_c.sort_values(by='similarity_score', ascending=False)

,product_name,product_review,similarity_score
4054,Galaxy Dry Hopped Fort Point Pale Ale,"11/2014 - Wow, this is amazing; incredible aro...",0.264906
2512,Abner,Tasted from can. Pours a hazy gold. Huge citru...,0.264906
5497,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,0.258199
4453,Peche 'n Brett,One of the 14 beers we have had in one session...,0.246183
16,Oude Geuze Golden Blend,2014 vintage; drank 6/22/19 @ Golgi Apparatus....,0.240772
...,...,...,...
2280,Hommage,"Blend 40 Season 17/18. Bottled on March 22, 20...",0.000000
2279,Hunahpu's Imperial Stout - Bourbon Barrel-Aged,2017 release (El Cat Round 7).A: Pours a visco...,0.000000
2278,Hunahpu's Imperial Stout - Bourbon Barrel-Aged,22 oz bomber gifted from Donald and poured int...,0.000000
2277,Wide Awake It's Morning,Dark with a rich creamy tan head. Smells of co...,0.000000


# Task D

In [388]:
# performing sentiment analysis
sia = SentimentIntensityAnalyzer()
df['scores'] = df['product_review'].apply(lambda review: sia.polarity_scores(review))
df['sentiment_score']  = df['scores'].apply(lambda score_dict: score_dict['compound'])

# create another version of the table for the output
df_d = df.copy()
df_d = df_d.drop(['clean_review', 'join_review', 'similarity_score'], axis=1)

# sort values according to the sentiment score
df_d.sort_values(by='sentiment_score', ascending=False)

,product_name,product_review,user_rating,scores,sentiment_score
5957,Mornin' Delight,An unbelievably huge thanks to Chris for this ...,4.56,"{'neg': 0.041, 'neu': 0.703, 'pos': 0.256, 'co...",0.9992
6073,Abraxas,16 oz. Cup (From On-Tap/Beer To Go)From: The A...,4.75,"{'neg': 0.0, 'neu': 0.731, 'pos': 0.269, 'comp...",0.9992
1828,Cable Car Kriek,Brown glass bottle served into a wine glass at...,4.42,"{'neg': 0.038, 'neu': 0.69, 'pos': 0.272, 'com...",0.9992
3070,No Rules,Jan 2021 New Years Day2019 vintage from pint g...,4.82,"{'neg': 0.028, 'neu': 0.683, 'pos': 0.289, 'co...",0.9991
14,The Peach,one of the beers i have been after for a long ...,4.45,"{'neg': 0.037, 'neu': 0.691, 'pos': 0.272, 'co...",0.9991
...,...,...,...,...,...
186,Ghost In The Machine - Double Dry-Hopped,Currently ranked #24 in the Top New England IP...,3.46,"{'neg': 0.168, 'neu': 0.731, 'pos': 0.102, 'co...",-0.9421
99,Leche Borracho,"On tap at Hollingshead’s in Orange, CA.Oh my s...",4.49,"{'neg': 0.287, 'neu': 0.609, 'pos': 0.104, 'co...",-0.9449
4278,Black Tuesday,Received from @2beerdogs in NBS BIF 10 – Thank...,3.94,"{'neg': 0.11, 'neu': 0.835, 'pos': 0.055, 'com...",-0.9584
4889,Fuzzy Baby Ducks,Poured out of a can into a tree house pint gla...,4.19,"{'neg': 0.318, 'neu': 0.631, 'pos': 0.051, 'co...",-0.9659


# Task E

In [389]:
# created another dataframe for aggregating the similarity scores and sentiment scores
output = df.groupby(['product_name'])['similarity_score', 'sentiment_score'].mean()

# calculated the evaluation scores
output['evaluation_score'] = output['similarity_score'] + output['sentiment_score']
output

,similarity_score,sentiment_score,evaluation_score
product_name,,,
A Deal With The Devil,0.011443,0.699264,0.710707
A Deal With The Devil - Double Oak-Aged,0.008051,0.841796,0.849847
Aaron,0.004800,0.829384,0.834184
Abner,0.036676,0.864592,0.901268
Abrasive Ale,0.018156,0.766700,0.784856
...,...,...,...
Westly,0.028731,0.761940,0.790671
Wide Awake It's Morning,0.019441,0.627304,0.646745
Zenne Y Frontera,0.042271,0.874504,0.916775


In [390]:
output.sort_values(by='evaluation_score', ascending=False)[0:3]

,similarity_score,sentiment_score,evaluation_score
product_name,,,
Cable Car Kriek,0.024023,0.930252,0.954275
Cable Car,0.024918,0.904408,0.929326
Genealogy Of Morals - Bourbon Barrel-Aged,0.015748,0.902328,0.918076


#### The top 3 products are 'Cable Car Kriek', 'Cable Car', 'Genealogy of Morals - Bourbon Barrel-Aged'.

# Task F - Part 1

### Cosine similarity using word vectors

In [391]:
# using word vectors to calculate the similarities instead of bag-of-words
nlp = en_core_web_lg.load()
def cosine_similarity(comment):
    base = nlp(comment)
    compare = nlp(input_attributes)
    return base.similarity(compare)

In [392]:
# join the words from the clean_review column
def join_words(comment):
    return " ".join(comment)
df_f['join_review'] = df_f['clean_review'].map(join_words)

In [394]:
# the three attributes we chose in Task B
attributes = ['fruity', 'balanced', 'robust']
input_attributes = " ".join(attributes)

# getting the similarity scores
df_f['similarity_score'] = df_f['join_review'].map(cosine_similarity)

In [395]:
# this output shows the similarity scores calculated using word vectors
output_df = df_f[['product_name', 'product_review', 'similarity_score']]

In [396]:
# sentiment analysis on the product_review (performed this again because we are using a different dataframe)
sia = SentimentIntensityAnalyzer()
df_f['scores'] = df_f['product_review'].apply(lambda review: sia.polarity_scores(review))
df_f['sentiment_score']  = df_f['scores'].apply(lambda score_dict: score_dict['compound'])
df_f.sort_values(by='sentiment_score', ascending=False)

# create another version of the table for the output
output_f = df.copy()
output_f = output_f.drop(['clean_review', 'join_review', 'similarity_score'], axis=1)

# sort values according to the sentiment score
#output_f.sort_values(by='sentiment_score', ascending=False)

In [397]:
# created another dataframe for aggregating the similarity scores and sentiment scores
output_f1 = df_f.groupby(['product_name'])['similarity_score', 'sentiment_score'].mean()

# calculated the evaluation scores
output_f1['evaluation_score'] = output_f1['similarity_score'] + output_f1['sentiment_score']
output_f1

,similarity_score,sentiment_score,evaluation_score
product_name,,,
A Deal With The Devil,0.637332,0.699264,1.336596
A Deal With The Devil - Double Oak-Aged,0.639584,0.841796,1.481380
Aaron,0.647567,0.829384,1.476951
Abner,0.656574,0.864592,1.521166
Abrasive Ale,0.671344,0.766700,1.438044
...,...,...,...
Westly,0.662809,0.761940,1.424749
Wide Awake It's Morning,0.637323,0.627304,1.264627
Zenne Y Frontera,0.682039,0.874504,1.556543


In [398]:
# top products
output_f1.sort_values(by='evaluation_score', ascending=False)[0:3]

,similarity_score,sentiment_score,evaluation_score
product_name,,,
Cable Car Kriek,0.658513,0.930252,1.588765
Flora Plum,0.680517,0.890092,1.570609
Zenne Y Frontera,0.682039,0.874504,1.556543


#### New top products are 'Cable Car Kriek', 'Flora Plum', and 'Zenne Y Frontera'.

# Task F - Part 2 (% of reviews)

In [399]:
attributes = ['fruity', 'balanced', 'robust']

# creating columns for each attribute
for attribute in attributes:
    df['contains_' + attribute] = 0

# replacing 0s with 1s for reviews that mention the attributes
for i in range(len(df['join_review'])):
    for attribute in attributes:
        if attribute in df['join_review'][i]:
            df['contains_' + attribute][i] = 1

In [400]:
# calculate the proportions of reviews that mention the attributes for each product
contain_df = df.groupby(['product_name'])['similarity_score','sentiment_score','contains_fruity', 'contains_balanced', 'contains_robust'].mean()
contain_df['evaluation_score'] = contain_df['similarity_score'] + contain_df['sentiment_score']

In [401]:
# merge two dataframes so that we can efficiently compare similarity scores calculated using two different methods
comparison_df = contain_df.merge(output_f1, left_on = 'product_name', right_on = 'product_name', suffixes=('_bow', '_vec'))

### Comparison between evaluation scores and % of mentions
### Sorted by bag-of-words evaluation scores

In [368]:
# sort the comparison dataframe using the bag-of-words evaluation score
comparison_df1 = comparison_df.drop(['similarity_score_bow','similarity_score_vec','sentiment_score_bow','sentiment_score_vec'], axis=1)
comparison_df1.sort_values(by='evaluation_score_bow', ascending=False)[0:3]

,contains_fruity,contains_balanced,contains_robust,evaluation_score_bow,evaluation_score_vec
product_name,,,,,
Cable Car Kriek,0.28,0.12,0.04,0.954275,1.588765
Cable Car,0.12,0.16,0.00,0.929326,1.514330
Genealogy Of Morals - Bourbon Barrel-Aged,0.04,0.12,0.04,0.918076,1.544946


### Comparison between evaluation scores and % of mentions
### Sorted by word vector evaluation scores

In [369]:
# sort the comparison dataframe using the word vector evaluation score
comparison_df1.sort_values(by='evaluation_score_vec', ascending=False)[0:3]

,contains_fruity,contains_balanced,contains_robust,evaluation_score_bow,evaluation_score_vec
product_name,,,,,
Cable Car Kriek,0.28,0.12,0.04,0.954275,1.588765
Flora Plum,0.20,0.16,0.00,0.912830,1.570609
Zenne Y Frontera,0.20,0.28,0.00,0.916775,1.556543


If we use word vectors for the cosine similarity calculation, we see that our top product (Cable Car Kriek) stays the same however, the other two products change. Flora Plum and Zenne Y Frontera are now second and third place respectively. Overall, as shown in the two comparison tables above, the proportion of each product's reviews mentioning the attribute is higher when we use word vectors to calculate cosine similarity.

# Task G

In [402]:
# calculate the average user ratings for each product
df_g1 = df_g.groupby(['product_name'])['user_rating'].mean()
df_g1 = pd.DataFrame(df_g1)

# extract the top 3 products
df_g1.sort_values(by='user_rating', ascending=False)[0:3]

,user_rating
product_name,
Chemtrailmix,4.77160
Vanilla Bean Assassin,4.74625
Blessed,4.74280


In [403]:
# extract the similarity scores, sentiment scores, evaluation scores, and proportions for the top 3
comp_df = comparison_df.loc[['Chemtrailmix', 'Vanilla Bean Assassin', 'Blessed']]
comp_df = comp_df.drop(['similarity_score_vec','sentiment_score_vec', 'evaluation_score_vec'], axis=1)
comp_df

,similarity_score_bow,sentiment_score_bow,contains_fruity,contains_balanced,contains_robust,evaluation_score_bow
product_name,,,,,,
Chemtrailmix,0.019334,0.791708,0.0,0.2000,0.0,0.811042
Vanilla Bean Assassin,0.006481,0.812306,0.0,0.0625,0.0,0.818787
Blessed,0.016703,0.739048,0.0,0.2000,0.0,0.755751


If we ignored the similarity and feature sentiment scores and simply chose the 3 products based on average user ratings, we got a completely different combination of products: 'Chemtrailmix', 'Vanilla Bean Assassin', and 'Blessed'. Using the similarity scores calculated with bag-of-words, these products will not meet the requirements of the user looking for recommendations. Although the sentiment scores are high, the similarity scores of the three products against the selected attributes are very low. This could mean that there are positive reviews on the three products based on other attributes. As shown in the table, the proportions of each of the attributes in the three products are very low. There are no mentions for each of the products about the 'Fruity' and 'Robust' attributes. Therefore, this set of recommendations will not satisfy the customer's needs.